# State by State Daily Ad Counts by Party - 7DMA

In [1]:
import pprint
import pickle
import pandas as pd
from datetime import datetime
import numpy as np
import string
from collections import Counter
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl

pd.set_option("display.max_columns", 2000)
pd.set_option("display.max_rows", 2000)
pd.set_option("display.precision", 2)
# pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# Using 'fivethirtyeight' style sheet

plt.style.use('fivethirtyeight')

In [3]:
# reading in cleaned up dataset

cand_date_df = pd.read_pickle('data/pres_sorted_with_sponsors_and_party.pickle')
cand_date_df.head()

,id,wp_identifier,network,location,program,program_type,start_time,end_time,archive_id,embed_url,sponsors,sponsor_types,race,cycle,subjects,candidates,type,message,date_created,ad_length,day_of_week,metro_area,state,subject_count,hour_aired,sponsor_class,party
start_time,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-08-01 01:19:59,261456,6035,FOXNEWSW,"San Francisco-Oakland-San Jose, CA",On the Record With Greta Van Susteren,news,2016-08-01 01:19:59,2016-08-01 01:21:04,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,2016.0,Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.0,Monday,San Francisco-Oakland-San Jose,National,1.0,1,R_PAC,R
2016-08-01 06:20:04,261450,6035,FOXNEWSW,"San Francisco-Oakland-San Jose, CA",Fox News Sunday With Chris Wallace,news,2016-08-01 06:20:04,2016-08-01 06:21:09,PolAd_HillaryClinton_4drmd,https://archive.org/embed/PolAd_HillaryClinton...,Reform America Fund,Super PAC,PRES,2016.0,Candidate Biography,Hillary Clinton,campaign,con,2016-09-21 10:40:12,65.0,Monday,San Francisco-Oakland-San Jose,National,1.0,6,R_PAC,R
2016-08-01 08:39:49,48222,4176,WCAU,"Philadelphia, PA",NBC10 News Today at 430am,news,2016-08-01 08:39:49,2016-08-01 08:40:21,PolAd_DonaldTrump_99k4w,https://archive.org/embed/PolAd_DonaldTrump_99k4w,Hillary for America,Multiple,PRES,2016.0,"Candidate Biography, Foreign Policy, Nuclear, ...",Donald Trump,campaign,mixed,2016-09-12 14:51:51,32.0,Monday,Philadelphia,PA,4.0,8,Clinton,D
2016-08-01 08:48:14,48395,4176,KYW,"Philadelphia, PA",Eyewitness News at 430,news,2016-08-01 08:48:14,2016-08-01 08:48:46,PolAd_DonaldTrump_99k4w,https://archive.org/embed/PolAd_DonaldTrump_99k4w,Hillary for America,Multiple,PRES,2016.0,"Candidate Biography, Foreign Policy, Nuclear, ...",Donald Trump,campaign,mixed,2016-09-12 14:51:51,32.0,Monday,Philadelphia,PA,4.0,8,Clinton,D
2016-08-01 09:09:45,378441,7459,MSNBCW,"San Francisco-Oakland-San Jose, CA",First Look,news,2016-08-01 09:09:45,2016-08-01 09:10:15,PolAd_HillaryClinton_4gvic,https://archive.org/embed/PolAd_HillaryClinton...,Make America Number One,Super PAC,PRES,2016.0,Candidate Biography,Hillary Clinton,campaign,con,2016-10-21 10:40:01,30.0,Monday,San Francisco-Oakland-San Jose,National,1.0,9,R_PAC,R


In [4]:
cand_date_df['state'].value_counts()

FL          12284
PA          11915
NV          10928
OH          10322
National     7217
NC           6722
IA           5284
NH           4963
CO           4689
WI           4065
AZ           3219
VA           1695
NY             46
Name: state, dtype: int64

In [5]:
# getting rid of NY

cand_date_df = cand_date_df[cand_date_df['state'] != 'NY']

## Pickling `cand_date_df` to generate visualizations

In [6]:
# cand_date_df.to_pickle('data/political_tv_ads_cleaned_for_visuals.pickle')

In [7]:
state_list = list(cand_date_df['state'].unique())
state_list

['National', 'PA', 'NV', 'NC', 'FL', 'NH', 'CO', 'OH', 'AZ', 'WI', 'VA', 'IA']

In [8]:
# creating a dictionary of DataSeries
# key = Series Name, value = Count of Ads

state_DataFrames = {}
party_list = ['D', 'R']

for state in state_list:
    for party_initial in party_list:
        state_DataFrames[str(state+'_'+party_initial)] = cand_date_df.copy()['party'][(cand_date_df['party'] == party_initial) & 
                                                                                      (cand_date_df['state'] == state)].resample('D').count()

In [9]:
state_DataFrames.keys()

dict_keys(['NV_D', 'FL_R', 'OH_R', 'VA_D', 'FL_D', 'NH_R', 'WI_D', 'IA_R', 'NC_D', 'AZ_D', 'National_R', 'PA_R', 'National_D', 'WI_R', 'CO_R', 'NC_R', 'NH_D', 'VA_R', 'PA_D', 'OH_D', 'NV_R', 'IA_D', 'CO_D', 'AZ_R'])

In [10]:
type(state_DataFrames['OH_R'])

pandas.core.series.Series

In [11]:
state_keys_list = sorted(list(state_DataFrames.keys()))
state_keys_list

['AZ_D',
 'AZ_R',
 'CO_D',
 'CO_R',
 'FL_D',
 'FL_R',
 'IA_D',
 'IA_R',
 'NC_D',
 'NC_R',
 'NH_D',
 'NH_R',
 'NV_D',
 'NV_R',
 'National_D',
 'National_R',
 'OH_D',
 'OH_R',
 'PA_D',
 'PA_R',
 'VA_D',
 'VA_R',
 'WI_D',
 'WI_R']

In [12]:
# Putting new DataSeries into a list -- need to do so in order to use pd.concat to join them together in a DataFrame

state_DataFrames_list = []

for item in state_keys_list:
    state_DataFrames_list.append(state_DataFrames[item])

In [13]:
# Making sure new list has all the dataframes

len(state_keys_list), len(state_DataFrames_list)

(24, 24)

In [14]:
# Using pd.concat to join individual series in a list in one DataFrame - harmonize start dates

state_party_df = pd.concat(state_DataFrames_list, axis=1, join='outer')
state_party_df.columns = state_keys_list
state_party_df.head(20)

,AZ_D,AZ_R,CO_D,CO_R,FL_D,FL_R,IA_D,IA_R,NC_D,NC_R,NH_D,NH_R,NV_D,NV_R,National_D,National_R,OH_D,OH_R,PA_D,PA_R,VA_D,VA_R,WI_D,WI_R
start_time,,,,,,,,,,,,,,,,,,,,,,,,
2016-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,23,NaN,NaN,18,6,NaN,NaN,NaN,NaN
2016-08-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,20,NaN,NaN,69,5,NaN,NaN,NaN,NaN
2016-08-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,17,NaN,NaN,81,4,NaN,NaN,NaN,NaN
2016-08-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,8,10,NaN,NaN,54,3,NaN,NaN,NaN,NaN
2016-08-05,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,12.0,NaN,1.0,NaN,10.0,NaN,14,6,NaN,NaN,35,0,NaN,NaN,NaN,NaN
2016-08-06,NaN,NaN,2.0,NaN,21.0,NaN,NaN,NaN,12.0,NaN,6.0,NaN,28.0,NaN,12,14,NaN,NaN,19,0,NaN,NaN,NaN,NaN
2016-08-07,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,2.0,NaN,2.0,NaN,5.0,1.0,6,3,5.0,NaN,19,2,NaN,NaN,NaN,NaN
2016-08-08,5.0,NaN,1.0,2.0,38.0,20.0,NaN,NaN,31.0,10.0,6.0,NaN,36.0,0.0,19,40,60.0,26.0,47,1,NaN,NaN,3.0,NaN
2016-08-09,3.0,1.0,2.0,2.0,54.0,15.0,NaN,NaN,24.0,17.0,12.0,1.0,69.0,2.0,35,34,81.0,33.0,84,2,NaN,NaN,2.0,NaN


In [15]:
# Now that all of the states have the same start dates -- fillna with zero then take the rolling averages

state_party_df.fillna(0, inplace=True)
state_party_df.head(20)

,AZ_D,AZ_R,CO_D,CO_R,FL_D,FL_R,IA_D,IA_R,NC_D,NC_R,NH_D,NH_R,NV_D,NV_R,National_D,National_R,OH_D,OH_R,PA_D,PA_R,VA_D,VA_R,WI_D,WI_R
start_time,,,,,,,,,,,,,,,,,,,,,,,,
2016-08-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,23,0.0,0.0,18,6,0.0,0.0,0.0,0.0
2016-08-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,20,0.0,0.0,69,5,0.0,0.0,0.0,0.0
2016-08-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21,17,0.0,0.0,81,4,0.0,0.0,0.0,0.0
2016-08-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8,10,0.0,0.0,54,3,0.0,0.0,0.0,0.0
2016-08-05,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,12.0,0.0,1.0,0.0,10.0,0.0,14,6,0.0,0.0,35,0,0.0,0.0,0.0,0.0
2016-08-06,0.0,0.0,2.0,0.0,21.0,0.0,0.0,0.0,12.0,0.0,6.0,0.0,28.0,0.0,12,14,0.0,0.0,19,0,0.0,0.0,0.0,0.0
2016-08-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,5.0,1.0,6,3,5.0,0.0,19,2,0.0,0.0,0.0,0.0
2016-08-08,5.0,0.0,1.0,2.0,38.0,20.0,0.0,0.0,31.0,10.0,6.0,0.0,36.0,0.0,19,40,60.0,26.0,47,1,0.0,0.0,3.0,0.0
2016-08-09,3.0,1.0,2.0,2.0,54.0,15.0,0.0,0.0,24.0,17.0,12.0,1.0,69.0,2.0,35,34,81.0,33.0,84,2,0.0,0.0,2.0,0.0


In [16]:
state_party_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2016-08-01 to 2016-11-08
Freq: D
Data columns (total 24 columns):
AZ_D          100 non-null float64
AZ_R          100 non-null float64
CO_D          100 non-null float64
CO_R          100 non-null float64
FL_D          100 non-null float64
FL_R          100 non-null float64
IA_D          100 non-null float64
IA_R          100 non-null float64
NC_D          100 non-null float64
NC_R          100 non-null float64
NH_D          100 non-null float64
NH_R          100 non-null float64
NV_D          100 non-null float64
NV_R          100 non-null float64
National_D    100 non-null int64
National_R    100 non-null int64
OH_D          100 non-null float64
OH_R          100 non-null float64
PA_D          100 non-null int64
PA_R          100 non-null int64
VA_D          100 non-null float64
VA_R          100 non-null float64
WI_D          100 non-null float64
WI_R          100 non-null float64
dtypes: float64(20), int64(4)
memory

In [17]:
state_party_df.shape

(100, 24)

In [18]:
# make sure these numbers match DataFrame below for '2016-08-07'

state_party_df['National_D']['2016-08-01':'2016-08-07'].sum() / 7

11.0

In [19]:
state_party_df['National_R']['2016-08-01':'2016-08-07'].sum() / 7

13.285714285714286

In [20]:
# Can apply the .rolling() to a whole dataframe -- which is convenient -- versus doing one series at a time
# Only time you get a NaN when there is not enough observations at the beginning of the dataset
# hence there will be 6 NaNs for each column -- see .info() below

state_party_df = state_party_df.rolling(7).mean()

In [21]:
state_party_df['2016-08-07':'2016-09-01']

,AZ_D,AZ_R,CO_D,CO_R,FL_D,FL_R,IA_D,IA_R,NC_D,NC_R,NH_D,NH_R,NV_D,NV_R,National_D,National_R,OH_D,OH_R,PA_D,PA_R,VA_D,VA_R,WI_D,WI_R
start_time,,,,,,,,,,,,,,,,,,,,,,,,
2016-08-07,0.00,0.00,0.29,0.00,3.71,0.00,0.00,0.00,3.71,0.00,1.29,0.00,6.29,0.14,11.00,13.29,0.71,0.00,42.14,2.86,0.00,0.00,0.00,0.00
2016-08-08,0.71,0.00,0.43,0.29,9.14,2.86,0.00,0.00,8.14,1.43,2.14,0.00,11.43,0.14,13.14,15.71,9.29,3.71,46.29,2.14,0.00,0.00,0.43,0.00
2016-08-09,1.14,0.14,0.71,0.57,16.86,5.00,0.00,0.00,11.57,3.86,3.86,0.14,21.29,0.43,16.43,17.71,20.86,8.43,48.43,1.71,0.00,0.00,0.71,0.00
2016-08-10,1.57,0.14,1.14,0.57,24.14,6.86,0.00,0.00,15.14,6.29,4.86,0.14,29.86,0.43,27.57,17.43,30.71,12.29,50.29,1.14,0.00,0.00,1.14,0.00
2016-08-11,2.29,0.14,1.86,0.57,32.86,10.29,0.00,0.00,18.86,9.57,6.29,0.14,35.71,0.43,35.86,19.14,39.00,16.57,56.00,0.71,0.00,0.00,2.00,0.00
2016-08-12,2.43,0.14,2.14,0.57,40.00,13.14,0.00,0.00,20.43,12.57,6.43,0.14,42.00,0.43,39.71,21.71,47.29,21.43,63.57,0.71,0.00,0.00,2.29,0.00
2016-08-13,2.71,0.14,2.14,0.57,40.57,13.57,0.00,0.00,21.57,13.43,6.71,0.14,42.43,0.43,41.71,20.14,50.00,22.43,67.57,0.71,0.00,0.00,2.43,0.00
2016-08-14,3.57,0.29,2.71,0.71,44.14,14.00,0.00,0.00,24.86,14.00,7.57,0.29,47.43,0.43,44.57,21.14,55.57,23.86,72.43,0.57,0.00,0.00,3.00,0.14
2016-08-15,3.57,0.29,3.29,0.43,48.14,14.43,0.00,0.00,25.86,14.43,8.43,0.29,52.43,0.43,45.14,18.00,58.86,24.43,76.57,0.43,0.00,0.00,3.00,0.14


In [22]:
state_party_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2016-08-01 to 2016-11-08
Freq: D
Data columns (total 24 columns):
AZ_D          94 non-null float64
AZ_R          94 non-null float64
CO_D          94 non-null float64
CO_R          94 non-null float64
FL_D          94 non-null float64
FL_R          94 non-null float64
IA_D          94 non-null float64
IA_R          94 non-null float64
NC_D          94 non-null float64
NC_R          94 non-null float64
NH_D          94 non-null float64
NH_R          94 non-null float64
NV_D          94 non-null float64
NV_R          94 non-null float64
National_D    94 non-null float64
National_R    94 non-null float64
OH_D          94 non-null float64
OH_R          94 non-null float64
PA_D          94 non-null float64
PA_R          94 non-null float64
VA_D          94 non-null float64
VA_R          94 non-null float64
WI_D          94 non-null float64
WI_R          94 non-null float64
dtypes: float64(24)
memory usage: 19.5 KB


## Pickling DataFrame to Use in a Separate Notebook for State by State Visualizations

In [24]:
# state_party_df.to_pickle('data/state_party_ad_counts_7DMA.pickle')